##Installation

In [ ]:
#install before running


#from derivative import dxdt


from google.colab import files
import numpy as np
import pandas as pd
#import pdflatex
import matplotlib as mpl

import matplotlib.pyplot as plt
#from mpl_toolkits import mplot3d
import matplotlib.gridspec as gridspec
#from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
from matplotlib import cm
import matplotlib.patches as mpatches
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import matplotlib.lines as mlines

from scipy.ndimage import median_filter,generic_filter
import scipy.ndimage as sc


from scipy.interpolate import splrep, BSpline
from scipy.interpolate import UnivariateSpline
from scipy import interpolate
from scipy import optimize
import scipy.interpolate as sci

from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)

from matplotlib.legend_handler import HandlerLine2D, HandlerTuple
from matplotlib.legend_handler import HandlerBase
#from matplotlib import collections



#import ot
from sklearn.neighbors import KernelDensity
import numpy.random as npr

##Laundauer model

###KL DIVERGENCE
Functions for the predicted cumulants and corrections for the marginal density of the position and the optimal protocol.

In [ ]:
#df_orig = pd.read_csv("ep_land4.csv",index_col=0)
path = "results_kl_land44.csv"
#"underdamped_kl_results_v1.csv"
data = pd.read_csv(path)#,index_col = 0)
df_orig = pd.DataFrame(data)
df = df_orig.rename({'ptx': 'rho'}, axis='columns')

#fill infs and nans with zeros
df.replace(np.inf, 0, inplace=True)
df.replace(np.nan, 0, inplace=True)


##PARAMETERS
Tf = 5 #2#4
epsilon = 0.2#np.sqrt(0.1)#0.2

g = 0
omega = 1
beta = 1

#decimal places for the time lookup.
dps =  4#
filter_delta = 20 #size of convolution filter to use

#tolerance to zero.
tol = 1e-10

#constants
def B_fn(Tf):
  return -((1+g)/Tf)*np.tanh(omega*Tf/2)*(omega*np.tanh(Tf) - 2*np.tanh(omega*Tf/2))/(omega*np.tanh(omega*Tf/2) - 2*np.tanh(Tf))

def A_fn(Tf):
  return (1+g)*(1 - (((omega**2 - 4)*(np.tanh(omega*Tf/2)*np.tanh(Tf)))/(omega*Tf*(omega*np.tanh(omega*Tf/2)-2*np.tanh(Tf)))))

T = Tf
A_minus_B = (1+g)*(1-((2/(omega*T))*(np.tanh(omega*T/2))))

A = A_fn(Tf)
B = B_fn(Tf)

alpha = np.sqrt((1+g)*A)

#clips values close to zero to prevent errors in logarithms
def zchop(a,tol = tol):
  #input: vector a
  #output: vector with values close to zero clipped.
  a[np.abs(a) < tol] = 0.0
  return a

#get values of t2
t2_vec = np.array(df.t.unique())
t2_vec.sort()

#compute mean and variance using all data
mean_0 = np.zeros(len(t2_vec))
var_0 = np.zeros(len(t2_vec))

for t2 in enumerate(t2_vec):
  #get q-axis
  q_temp = df[df.t == t2[1]].x.to_numpy()
  #get rho
  rho_temp = df[df.t == t2[1]].rho.to_numpy()
  #compute first order mean and var as functions of t2
  mean_0[t2[0]] = np.trapz(q_temp*rho_temp,q_temp)
  var_0[t2[0]] = np.trapz((q_temp**2)*rho_temp,q_temp)


In [ ]:
#cut the x-axis so that each is of the same length.
#This should only remove where there is no mass, ie rho = 0
xmin = -5
xmax = 5

for t2 in t2_vec:
  xmin = np.maximum(xmin,np.min(df[df.t==t2].x))
  xmax = np.minimum(xmax,np.max(df[df.t==t2].x))

#delete edge rows
df.drop(df[df.x < xmin].index, inplace=True)
df.drop(df[df.x > xmax].index, inplace=True)



In [ ]:


#get qaxis
q_axis = df[df.t == 0].x.to_numpy()

#function to get rho at time t0.
def rho(t0):
  t2 = round(t0*(epsilon**2),dps)
  rho_temp = df[df.t==t2].rho.to_numpy()
  return zchop(rho_temp)

#get all xcoords where there is probability mass
def get_rhomask(t0):
  return np.where(rho(t0)>0)

#generate vecotr of t0 from values of t2
times_t0 = np.round(t2_vec/(epsilon**2),dps)


In [ ]:



def b(t0,g=0):
  omega = 1#np.sqrt((1+g)/g)

  denom1 =np.cosh(T)*np.sinh(omega*T) -(2*np.sinh(T)*np.cosh(omega*T)+2*np.sinh(T))/omega

  num3 = np.sinh(omega*(T-t0))*np.exp(-T)
  num4 = np.sinh(omega*t0)*np.exp(T) - np.sinh(omega*T)*np.exp(2*t0 - T)
  return (1/denom1)*(num3+num4)

def a_minus_b2(t0,g=0):
  omega = 1#np.sqrt((1+g)/g)

  return 1 -(np.cosh(omega*t0)+np.cosh(omega*(t0-T)))/(1+np.cosh(T*omega))

def a(t0,g=0):

  return a_minus_b2(t0,g) + b(t0,g)

def b_dot(t0,g=0):
  omega = 1#np.sqrt((1+g)/g)

  denom1 =np.cosh(T)*np.sinh(omega*T) -(2*np.sinh(T)*np.cosh(omega*T)+2*np.sinh(T))/omega

  num3 = -omega*np.cosh(omega*(T-t0))*np.exp(-T)
  num4 = omega*np.cosh(omega*t0)*np.exp(T) - 2*np.sinh(omega*T)*np.exp(2*t0 - T)
  return (1/denom1)*(num3+num4)
  """
  omega = np.sqrt((1+g)/g)

  denom1 = omega*np.cosh(T) - 2*(np.sinh(T)/np.tanh(omega*T/2))
  denom2 = np.sinh(omega*T)

  num1 = omega*np.sinh(omega*t0) - 2*np.exp(2*t0)
  num2 = omega*(np.exp(2*T) - np.cosh(omega*T))*np.cosh(omega*t0)

  return omega*np.exp(-T)*((num1/denom1) + (num2/denom1)/denom2)
  """

def a_dot(t0,g=0):
  omega = 1#np.sqrt((1+g)/g)

  term1 = -(np.sinh(omega*t0)+np.sinh(omega*(t0-T)))/(1+np.cosh(T*omega))

  return b_dot(t0,g) + omega*term1

#mean and variances
def mean_t0(t0):

  t2 = round(t0*(epsilon**2),dps)
  index_mean = np.where(t2_vec==t2)[0][0]

  return mean_0[index_mean]

def var_t0(t0):

  t2 = round(t0*(epsilon**2),dps)
  index_mean = np.where(t2_vec==t2)[0][0]

  return var_0[index_mean]

#function to get sigma
def sigma(t0):

  t2 = round(t0*(epsilon**2),dps)
  sigtemp = df[df.t==t2].sigma.to_numpy()

  return sigtemp


In [ ]:
#derivative functions
def dsigma(t0):
  sigma_temp = sigma(t0)
  idx = get_rhomask(t0)

  sigma_vals = sigma_temp[idx]
  q_vals = q_axis[idx]

  dsig = np.gradient(sigma_vals,q_vals)
  dsigout = generic_filter(dsig,sc.median,filter_delta,mode = "nearest")

  dtemp = np.zeros(len(q_axis))
  dtemp[idx] = dsigout

  return dtemp

def dfun(vals,qs):

  dfun = np.gradient(vals,qs)

  return generic_filter(dfun,sc.median,40,mode = "nearest")

def dlogrho(t0):
  logrho = np.log(rho(t0))

  #get rid of nans
  #interpolate only on non-zero vals of rho
  idx = get_rhomask(t0)
  logrho_temp = logrho[idx]
  q_axis_temp = q_axis[idx]

  #differentiate and filter without edges
  dlogrho = np.gradient(logrho_temp,q_axis_temp)
  filter_dlogrho = generic_filter(dlogrho,
                 sc.median,filter_delta,mode="nearest")

  #put back into right place
  dlogout = np.zeros(len(q_axis))
  dlogout[idx] = filter_dlogrho

  return dlogout

def drho(t0):
  rho_vals = rho(t0)

  #interpolate only on non-zero vals of rho
  idx = get_rhomask(t0)
  rho_vals_temp = rho_vals[idx]
  q_axis_temp = q_axis[idx]

  drho = np.gradient(rho_vals_temp,q_axis_temp)
  #set values outside of range to zero to prevent extrapolation error
  drho_vals = np.zeros(len(q_axis))
  drho_vals[idx] = generic_filter(drho,sc.median,filter_delta,mode="constant")

  return drho_vals

def rho_dsigma_alpha_rho(t0):

  rho_dsigma_alpha_rho = rho(t0)*dsigma(t0) + alpha*drho(t0)

  #fill nan with zero
  rho_dsigma_alpha_rho[np.isnan(rho_dsigma_alpha_rho)] = 0

  return rho_dsigma_alpha_rho

def dsigma_alpha_rho(t0):

  dsigma_alpha_rho = dsigma(t0) + alpha*dlogrho(t0)

  #fill nan with zero
  dsigma_alpha_rho[np.isnan(dsigma_alpha_rho)] = 0

  return dsigma_alpha_rho

def rho_ddsigma_alpha_rho(t0):

  #get drho
  drhotemp = dlogrho(t0)

  #remove zeros first
  idx = get_rhomask(t0)
  ddrhotemp = drhotemp[idx]
  q_temp = q_axis[idx]
  ddlogrho = np.gradient(ddrhotemp,q_temp)

  ddlogrho = generic_filter(ddlogrho,sc.mean,filter_delta,mode="nearest")

  #get ddsigma
  dsigtemp = dsigma(t0)
  dsig_vals = dsigtemp[idx]
  ddsigtemp = np.gradient(dsig_vals,q_temp)
  #dfun(dsigtemp,q_axis)
  #ddsig_add = ddsigtemp[~np.isnan(drhotemp)]

  temp_out = alpha*ddlogrho + generic_filter(ddsigtemp,sc.mean,filter_delta,mode="nearest")

  temp_vals_out = np.zeros(len(q_axis))
  temp_vals_out[idx] = temp_out

  output_vals = temp_vals_out*rho(t0)
  return generic_filter(output_vals,sc.mean,size=filter_delta,mode="constant")

In [ ]:

#compute cumulants and other functions
def kappa(t0): #mu dot 1
  integral = rho(t0) * dsigma(t0)
  return -np.trapz(integral,q_axis)

##
def f11(t0):

  rho_vals = rho(t0)

  coeff1 = -a(t0)/A
  num1 = rho_dsigma_alpha_rho(t0)

  coeff2 = rho_vals *(B*a(t0) - A*b(t0))/(A*A_minus_B)
   #(A-B))

  return coeff1*num1 + coeff2*kappa(t0)

##
def calculate_df11(t0):
  drho_vals = drho(t0)

  coeff1 = -a(t0)/A
  num1 = (drho_vals*dsigma_alpha_rho(t0)) + rho_ddsigma_alpha_rho(t0)

  num1[np.isnan(num1)] = 0
  num1[np.isinf(num1)] = 0

  coeff2 = (B*a(t0) - A*b(t0))/(A*A_minus_B)

  df11_out = coeff1*num1 + coeff2*(kappa(t0)*drho_vals)

  return df11_out


In [ ]:
#coefficients of the gradient of df11
def coeff1_df11(t0):
  coeff1 = -a(t0)/A
  return coeff1

def coeff2_df11(t0):
  coeff2 = (B*a(t0) - A*b(t0))/(A*A_minus_B)
  return coeff2

#compute f02 at t0 and fixed t2.
def f02_new(t0,g=g,T=T):

  #fetch the constants in t2
  t2_term1 = drho(t0)*dsigma_alpha_rho(t0) + rho_ddsigma_alpha_rho(t0)
  t2_termrho = drho(t0)*kappa(t0)

  term1 = -g*(coeff1_df11(t0)*t2_term1 + coeff2_df11(t0)*t2_termrho)

  int_limit = np.where(times_t0==t0)[0][0] + 1
  coeff1_df11_temp = [coeff1_df11(t)*t2_term1 for t in times_t0]
  coeff2_df11_temp = [coeff2_df11(t)*t2_termrho for t in times_t0]

  coeff1_int1 = (t0/T)*np.trapz(coeff1_df11_temp,times_t0,axis=0)
  coeff2_int1 = (t0/T)*np.trapz(coeff2_df11_temp,times_t0,axis=0)

  coeff1_int2 = np.trapz(coeff1_df11_temp[0:int_limit],times_t0[0:int_limit],axis=0)
  coeff2_int2 = np.trapz(coeff2_df11_temp[0:int_limit],times_t0[0:int_limit],axis=0)

  return term1 + (1+g)*(coeff1_int1 + coeff2_int1 - (coeff1_int2 + coeff2_int2))


In [ ]:


def conditional_mom_mean(t0): ##conditional momentum mean E(p|q)
  return epsilon*f11(t0)

def mom_mean(t0):
  return epsilon*(a(t0)-b(t0))*kappa(t0)/A_minus_B

#correction to the position marginal distribution
def pos_correction(t0):
  return rho(t0) + f02_new(t0)*(epsilon**2)

#linear position cumulant
#position mean
def linear_position_cumulant(t0):
  term1 = mean_t0(t0) + kappa(t0)*(epsilon**2)*g*(a(t0)-b(t0))/(A-B)
  coeff2 = (epsilon**2)*(1+g)*kappa(t0)/(A-B)
  int_limit = np.where(times_t0==t0)[0][0] + 1
  ab_temp = [a(t)-b(t) for t in times_t0]
  int1 = (t0/Tf)*np.trapz(ab_temp,times_t0,axis =0)
  int2 = np.trapz(ab_temp[0:int_limit],times_t0[0:int_limit],axis =0)

  return term1 - coeff2*(int1 - int2)

#Cross correlation
def script_k(t0):#varsigma dot
  temp_vals = q_axis*rho_dsigma_alpha_rho(t0)
  return -np.trapz(temp_vals,q_axis) - kappa(t0)*mean_t0(t0)

def cross_correlation(t0):
  return epsilon*a(t0)*script_k(t0)/A

#position process variance
def position_variance(t0):
  term1 = var_t0(t0) - (mean_t0(t0)**2)
  term2 = 2*(epsilon**2)*g*a(t0)*script_k(t0)/A

  coeff3 = -2*script_k(t0)*(epsilon**2)*(1+g)/A

  int_limit = np.where(times_t0==t0)[0][0] + 1
  a_temp = [a(t) for t in times_t0]

  int1 = (t0/Tf)*np.trapz(a_temp,times_t0,axis =0)
  int2 = np.trapz(a_temp[0:int_limit],times_t0[0:int_limit],axis =0)
  return term1 + term2 + coeff3*(int1 - int2)

#momentum variance
def momentum_variance(t0):

  term1 = 1 - ((kappa(t0)*epsilon*a(t0)/A)**2)

  int1 = np.trapz(rho_ddsigma_alpha_rho(t0),q_axis)/A
  int_limit = np.where(times_t0==t0)[0][0] + 1

  aexp_temp = [int1*a(t)*np.exp(-2*(t0-t)) for t in times_t0]
  int2 = np.trapz(aexp_temp[0:int_limit],times_t0[0:int_limit],axis =0)

  term2 = 2*(epsilon**2)*int2

  sq_temp = ((dsigma_alpha_rho(t0))**2)*rho(t0)

  term3 = ((epsilon*a(t0)/A)**2)*np.trapz(sq_temp,q_axis)

  return term1 + term2 + term3


In [ ]:
def calculate_distribution(t0):
  rvals = rho(t0)+ (epsilon**2)*f02_new(t0)

  #find normalising factor
  norm_factor = np.trapz(np.abs(rvals),q_axis)

  return rvals / norm_factor

def calculate_optimal_drift(t0):

  coeff1 = (a_dot(t0) + a(t0))/A

  dlog_rho = dlogrho(t0)

  term1 = (alpha*coeff1 - 1)*dlog_rho
  term2 = coeff1*dsigma(t0)

  coeff3 = kappa(t0)/(A*(A-B))
  term3 = (B*a_dot(t0) - A*b_dot(t0)) + (B*a(t0) - A*b(t0))
  opt_drift = term1 + term2 - coeff3*term3

  opt_drift[np.isnan(opt_drift)] = 0
  opt_drift[np.isinf(opt_drift)] = 0

  return -opt_drift


In [ ]:
#compute the pdfs and drifts for each t0 and adds them to the dataframe
df['UDpdf'] = 1
df['UDdrift'] = 1
df['smooth_UDpdf'] = 1

for t0 in times_t0:
  t2 = round(t0*(epsilon**2),dps)
  new_vals = calculate_distribution(t0)
  smooth_dist = generic_filter(new_vals,sc.mean,100,mode="constant")
  drift_vals = calculate_optimal_drift(t0)
  df.loc[df[df.t==t2].index,"UDpdf"] = new_vals
  df.loc[df[df.t==t2].index,"UDdrift"] = drift_vals
  df.loc[df[df.t==t2].index,"smooth_UDpdf"] = smooth_dist

#function to get sigma
def distribution(t0):
  t2 = round(t0*(epsilon**2),dps)
  dist = df[df.t==t2].UDpdf.to_numpy()
  return dist

#function to get sigma
def optimal_drift(t0):
  t2 = round(t0*(epsilon**2),dps)
  drift_temp = df[df.t==t2].UDdrift.to_numpy()
  return drift_temp


#function to get sigma
def smooth_distribution(t0):
  t2 = round(t0*(epsilon**2),dps)
  drift_temp = df[df.t==t2].smooth_UDpdf.to_numpy()
  return drift_temp


<ipython-input-7-5da9f27e2f42>:24: RuntimeWarning: divide by zero encountered in log
  logrho = np.log(rho(t0))
<ipython-input-12-42578de61cfa>:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0. 0. 0. ... 0. 0. 0.]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[df[df.t==t2].index,"UDpdf"] = new_vals
<ipython-input-12-42578de61cfa>:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[3.63560743 3.63560743 3.63560743 ... 3.63560743 3.63560743 3.63560743]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[df[df.t==t2].index,"UDdrift"] = drift_vals
<ipython-input-12-42578de61cfa>:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0. 0. 0. ... 0. 0. 0.]' has dtype incompatible with i

In [ ]:
df.to_csv("underdamped_kl_results_v3.csv")

###plots

In [ ]:
##make the plots

titlepad = 5
titlex = 0.15
titley = 0.85
titlezorder = 1000
titles = ["(a)","(b)","(c)","(d)","(e)","(f)","(g)","(h)","(i)","(j)","(k)","(l)","(m)","(n)","(o)","(p)","(q)"]
time_titles = ["$\mathrm{t} = 0$","$\mathrm{t} = 0.25\mathrm{t}_f$",
               "$\mathrm{t} = 0.4\mathrm{t}_f$","$\mathrm{t} = 0.5\mathrm{t}_f$",
               "$\mathrm{t} = 0.6\mathrm{t}_f$","$\mathrm{t} = 0.7\mathrm{t}_f$",
               "$\mathrm{t} = 0.8\mathrm{t}_f$","$\mathrm{t} = 0.95\mathrm{t}_f$",
               "$\mathrm{t} = \mathrm{t}_f$"]

#dist title location
disttitlex = -2.8
disttitley = 0.52

suptitlex = 0.07
suptitley = 1.1

dashlinezorder = 10

#alpha for filled shapes
shadingalpha = 1

# Plotting the graphs)
#COLORS
c2 = "#a6cee3" #lightblue
c3 = "orange" #"#33a02c" #dark green
c1 = "#1f78b4" #darkblue
c4 = "#b2df8a" #light green


#set ylim for plot
ymin = -30
ymax = 20

#xlims
xlimmax = 3
xlimmin = -3

lw = 3

#fontsizes
xmax = 5
fontsize = 22
fontsizeticks = 18
fontsizetitles = 22

def format_axes(ax,fontsize):

  ax.set_xlim((0,T))
  ax.set_xlabel(r"$\mathrm{t}$",fontsize = fontsizetitles)

  #ax.set_xticklabels(labels = [0,1,2,3,4,5],fontsize=10)
  ax.tick_params(axis='y', labelsize=fontsizeticks)
  ax.tick_params(axis='x', labelsize=fontsizeticks)
  return ax

def format_drift(ax):

  #ax.yaxis.tick_right()
  ax.tick_params(axis='x', labelsize=fontsizeticks)
  ax.tick_params(axis='y', labelsize=fontsizeticks)

  ax.set_ylim((-30,30))
  ax.set_xlim((xlimmin,xlimmax))
  ax.set_xlabel(r"$\mathrm{q}$",fontsize= fontsize,labelpad=7)

##plot set-up
def format_dist_axes(ax):

  #ax.patch.set_alpha(0)
  #ax.yaxis.tick_right()
  ax.tick_params(axis='x', labelsize=fontsizeticks)
  ax.tick_params(axis='y', labelsize=fontsizeticks)
  #ax.tick_params(labeltop='off', labelright='off')

  ax.set_ylim((-0.07,0.65))
  ax.set_xlim((xlimmin,xlimmax))
  ax.spines['bottom'].set_zorder(1000)

  #ax.set_xticks([])
  #ax.set_xticklabels([])
  #ax.set_xlabel(r"$\mathrm{q}$",fontsize= fontsize)

def cleaner(arr,t0):

  #masks nans and infs and returns a pair for plotting
  #copy q axis
  plotq = np.copy(q_axis)

  masknan = get_rhomask(t0)

  #this function removes nan's and the end points which come from the truncation of the gradients
  arr = arr[np.min(masknan)+20:np.max(masknan)-20]
  plotq = plotq[np.min(masknan)+20:np.max(masknan)-20]

  return plotq, arr
  #generic_filter(arr,sc.median,1,mode="constant")



#exact boundary conds
p_initial = np.exp(-(q_axis-1)**4)
p_final = np.exp(-(((q_axis**2 - 1)**2)))
pi_norm = np.abs(np.trapz(p_initial,q_axis))
pf_norm = np.abs(np.trapz(p_final,q_axis))

In [ ]:
#plot params


def plot_pdf_nucleation(tcurr,title,labels,loc):
  '''This functions plots a distribution and labels it

  input:
  -tcurr: the time in terms of t0 to plot
  -title: title of the graph (if using)
  -label: label of the panel
  -loc: location as a matplotlib subplot code (or gridspec)

  output:
  -plot in location given by loc with the overdamped(in orange)
  and underdamped (in blue) of the marginal density of the position
  '''

  # t0 distribution
  plt.subplot(loc)
  plt.title(title, loc = "center", fontsize=fontsizetitles)

  plt.plot(q_axis,rho(tcurr),color="orange",lw=4)
  plt.plot(q_axis,distribution(tcurr),color=c1,lw=3,  label =r"$\mathrm{t} = 0$",zorder = 10000)

  ax = plt.gca()
  format_dist_axes(ax)
  #ax.text(s = labels[0],fontweight ="bold",fontsize = fontsizetitles,x = disttitlex, y =disttitley,zorder = titlezorder)

  #ax.set_xticklabels([])
  ax.tick_params(axis='y', labelsize=fontsizeticks)


In [ ]:
plt.figure(figsize=(15,6))
plot_pdf_nucleation(0,"$\mathrm{t} = 0$",None,151)
ax0 = plt.gca()
ax0.fill_between(q_axis,p_initial/pi_norm,color = c2,alpha = shadingalpha)

plot_pdf_nucleation(0.5*T,"$\mathrm{t} = 0.5\ \mathrm{t}_{\mathrm{f}}$",None,152)
ax1 = plt.gca()
ax1.set_yticklabels([])
plot_pdf_nucleation(0.7*T,"$\mathrm{t} = 0.7\ \mathrm{t}_{\mathrm{f}}$",None,153)
ax1 = plt.gca()
ax1.set_yticklabels([])
plot_pdf_nucleation(0.85*T,"$\mathrm{t} = 0.85\ \mathrm{t}_{\mathrm{f}}$",None,154)
ax1 = plt.gca()
ax1.set_yticklabels([])
plot_pdf_nucleation(T,"$\mathrm{t} = \mathrm{t}_{\mathrm{f}}$",None,155)

plt.tight_layout()
ax = plt.gca()
ax.fill_between(q_axis,p_final/pf_norm,color = c2,alpha = shadingalpha)
ax.set_yticklabels([])

#make legend
l0 = mlines.Line2D([], [], color=c1, lw = lw)
l1 = mlines.Line2D([], [], color="orange", lw = lw)
p0 = mpatches.Patch(color = c2, alpha=shadingalpha)

ax.legend([l0,l1,p0],["Underdamped","Overdamped","Assigned Boundary Conditions"],
              prop = {"size": fontsizetitles },
              ncol = 4,
              bbox_to_anchor=(-4.8, -0.05),
              frameon = False)


plt.savefig("KL_LAND_PDFS.png")
#plt.savefig("KL_LAND_PDFS.pdf", format="pdf")
#plt.savefig("test1.eps", format="eps")

plt.close()

In [ ]:
#plot params

#fontsizes
xmax = 5
fontsize = 28
fontsizeticks = 22
fontsizetitles = 28

#set ylim for plot
ymin = -30
ymax = 30

def plot_pair(tcurr,title,labels,gs,locy):
  # t0 distribution
  plt.subplot(gs[0,locy])
  plt.title(title, loc = "center", fontsize=fontsizetitles)

  plt.plot(q_axis,rho(tcurr),color=c3,lw=4)
  plt.plot(q_axis,distribution(tcurr),color=c1,lw=3, label =r"$\mathrm{t} = 0$",zorder = 10000)

  #plt.plot(q_axis,rho(tcurr),color=c3,lw=4)
  #plt.plot(q_axis,generic_filter(distribution(tcurr),sc.mean,100,mode="constant"),color=c1,lw=3,  label =r"$\mathrm{t} = 0$",zorder = 10000)


  ax = plt.gca()
  format_dist_axes(ax)
  ax.text(s = labels[0],fontsize = fontsizetitles,x = disttitlex, y =disttitley,zorder = titlezorder)

  ax.set_xticklabels([])
  ax.tick_params(axis='y', labelsize=fontsizeticks)

  drift0 = plt.subplot(gs[1, locy])
  plot_data = cleaner(optimal_drift(tcurr),tcurr)

  #this just removes the areas of low statistics rho < tol
  qseries = plot_data[0]
  yseries = plot_data[1]
  sigma_data = cleaner(dsigma(tcurr),tcurr)
  sigmaseries = -sigma_data[1]

  ax0 = plt.gca()
  format_drift(ax0)
  ax0.text(s = labels[1],fontsize = fontsizetitles,x = disttitlex, y =25, zorder = titlezorder)

  #series1a, = ax0.plot(qseries,yseries,color = c1,lw=lw,label = r"Underdamped",alpha=0.5,zorder = 100)
  #series1b, = ax0.plot(qseries,sigmaseries,color = c3,lw=lw,label = r"Overdamped",alpha=0.5)
  series1c, = ax0.plot(qseries,generic_filter(yseries,sc.mean,100,mode="nearest"),color = c1,lw=lw,label = r"Underdamped",zorder = 10000)
  series1d, = ax0.plot(qseries,generic_filter(sigmaseries,sc.mean,100,mode="nearest"),color = c3,lw=lw,label = r"Underdamped",zorder = 10000)

  if locy ==0:
    ax.set_ylabel(r'$\mathrm{f}_{\mathrm{t}}(\mathrm{q})$',fontsize = fontsizetitles,labelpad= 7)
    ax0.set_ylabel(r'$-\partial U_{\mathrm{t}}(\mathrm{q})$',fontsize = fontsizetitles,labelpad= -5)
    if gs == gs0:
      #for edges
      ax.fill_between(q_axis,p_initial/pi_norm,color = c2,alpha = shadingalpha)
  else:
    ax0.set_yticklabels([])
    ax.set_yticklabels([])

  if locy ==-1 and gs == gs1:
    ax.fill_between(q_axis,p_final/pf_norm,color = c2,alpha = shadingalpha)


In [ ]:

#set up the gridspec

fig = plt.figure(figsize = (18,24)) #figsize = (width,height)

gs = gridspec.GridSpec(2, 1, height_ratios=[1,1],hspace=0.2)
gs0 = gridspec.GridSpecFromSubplotSpec(2, 5, height_ratios=[1,2], subplot_spec=gs[0], hspace=0.1, wspace=0.05)
gs1 = gridspec.GridSpecFromSubplotSpec(2, 5, height_ratios=[1,2], subplot_spec=gs[1], hspace=0.1, wspace=0.05)

plot_pair(0,"$\mathrm{t} = 0$",["(a)","(f)"],gs0,0)
plot_pair(0.25*T,"$\mathrm{t} = 0.25\ \mathrm{t}_f$",["(b)","(g)"],gs0,1)
plot_pair(0.5*T,"$\mathrm{t} = 0.5\ \mathrm{t}_f$",["(c)","(h)"],gs0,2)
plot_pair(0.6*T,"$\mathrm{t} = 0.6\ \mathrm{t}_f$",["(d)","(i)"],gs0,3)
plot_pair(0.7*T,"$\mathrm{t} = 0.7\ \mathrm{t}_f$",["(e)","(j)"],gs0,-1)

plot_pair(0.8*T,"$\mathrm{t} = 0.8\ \mathrm{t}_f$",["(k)","(p)"],gs1,0)
plot_pair(0.85*T,"$\mathrm{t} = 0.85\ \mathrm{t}_f$",["(l)","(q)"],gs1,1)
plot_pair(0.9*T,"$\mathrm{t} = 0.9\ \mathrm{t}_f$",["(m)","(r)"],gs1,2)
plot_pair(0.95*T,"$\mathrm{t} = 0.95\ \mathrm{t}_f$",["(n)","(s)"],gs1,3)
plot_pair(T,"$\mathrm{t} = \mathrm{t}_f$",["(o)","(t)"],gs1,-1)

#make legend
l0 = mlines.Line2D([], [], color=c1, lw = lw)
l1 = mlines.Line2D([], [], color=c3, lw = lw)
p0 = mpatches.Patch(color = c2, alpha=shadingalpha)

plt.legend([l0,l1,p0],["Underdamped","Overdamped","Assigned Boundary Conditions"],
              prop = {"size": fontsizeticks },
              ncol = 3,
              bbox_to_anchor=(0.4, -0.1),
              frameon = False)


#plt.savefig("kl_land_drift_v38.eps")
#plt.savefig("kl_land_drift_v38.pdf")
plt.savefig("kl_land_drift_v39.png")
plt.close()

###cumulant plots

In [ ]:
#Cumulant plots parameters
#df = pd.read_csv("kl_gauss_T2_v8.csv")
ti = 0#np.min(times_t0)
tf = T#np.max(times_t0)

tlim = (ti,tf)

####KL gauss
xmax = tf#
fontsize = 22
fontsizeticks = 18
fontsizetitles = 22

titlepad = 10
xtitle = 0.07
ytitle = 0.88
titlezorder = 1000


lw = 3

# Data for the graphs
x = df.t
titlepad = 5

In [ ]:
#GAUSSIAN PLOT

# Create a 2x3 grid with different widths for the bottom row
gs = gridspec.GridSpec(2, 6, width_ratios=[1, 1, 1, 1,1,1], height_ratios=[1, 1])

# Plotting the graphs
plt.figure(figsize=(15, 8))

#dashstyle
dashstyle = "dashed"#(0,(5,10))
dashstyle2 = "dotted"

# pos mean
plt.subplot(gs[0, 0:2])
plt.title('(a)',fontweight = "bold",fontsize = fontsizetitles,x = xtitle,y = ytitle,pad = titlepad,zorder=titlezorder)
plt.plot(x, df.p_pos_mean,color = c1,lw=lw)
plt.plot(x, df.pos_mean,linestyle = dashstyle,color = c2,lw=lw)
ax = format_axes(plt.gca(),fontsize)
ax.set_ylabel('Position Mean',fontsize = fontsizetitles)

# pos var
plt.subplot(gs[0, 2:4])
plt.title('(b)',fontweight = "bold",fontsize = fontsizetitles,x = xtitle,pad = titlepad ,y = ytitle,zorder=titlezorder)
plt.plot(x, df.p_pos_var,color =c1,lw=lw)
plt.plot(x, df.pos_var,linestyle = dashstyle,color = c2,lw=lw)

ax = plt.gca()
ax = format_axes(plt.gca(),fontsize)
ax.set_ylabel('Position Variance',fontsize = fontsizetitles)

###cross correlation
plt.subplot(gs[0, 4:])
plt.plot(x, df.p_cc,color = c1,lw=lw)
plt.plot(x, df.cc,linestyle = dashstyle,color = c2,lw=lw)

plt.title('(c)',fontweight = "bold",fontsize = fontsizetitles,x = xtitle,pad = titlepad ,y = ytitle,zorder=titlezorder)

ax = plt.gca()
ax.set_xlim((0,xmax))
ax = format_axes(plt.gca(),fontsize)
ax.set_ylabel('Cross Correlation',fontsize = fontsizetitles)

# momvar
mom_var = plt.subplot(gs[1, 3:])
plt.plot(x, df.p_mom_var,color = c1,lw=lw)
plt.plot(x, df.mom_var,linestyle = dashstyle,color = c2,lw=lw)
plt.title('(e)',fontweight = "bold",fontsize = fontsizetitles,x = xtitle*(2/3), pad = titlepad ,y = ytitle,zorder=titlezorder)

ax = plt.gca()
ax = format_axes(plt.gca(),fontsize)
ax.set_ylabel('Momentum Variance',fontsize = fontsizetitles)


plt.subplot(gs[1, :3])
plt.title('(d)',fontweight = "bold",fontsize = fontsizetitles,x = xtitle*(2/3),pad = titlepad ,y = ytitle,zorder=titlezorder)
ax = plt.gca()
ax = format_axes(plt.gca(),fontsize)

pert1, = ax.plot(x, df.p_mom_mean,label =r"Perturbative",color = c1,lw=lw)
npert1, = ax.plot(x, df.mom_mean,label =r"Non-Perturbative",linestyle = dashstyle,color = c2,lw=lw)

ax.set_ylabel('Momentum Mean',fontsize = fontsizetitles)

##make custom looking patch objects.???

# Create a legend for the first line.
first_legend = ax.legend([(pert1),(npert1)], ["Perturbative","Non Perturbative"],
                         handler_map={tuple: HandlerTuple(ndivide=None)},
                         prop = {"size": fontsizetitles }, loc='lower center',frameon=False)#,handlelength=1.5)

# Add the legend manually to the Axes.
ax.add_artist(first_legend)

# Adjust layout for better spacing
plt.tight_layout()

# save the plot
plt.savefig("kl_gauss_cumulants.eps", format="eps")
plt.savefig("kl_gauss_cumulants.pdf", format="pdf")
plt.savefig("kl_gauss_cumulants.png", format="png")
plt.close()

In [ ]:
#cumulants

filename = "kl_land_cumulants.eps"
format_type = "eps"


# Plotting the graphs
plt.figure(figsize=(15, 8))#, constrained_layout=True)

# Create a 1x5 grid with different widths for the bottom row
gs_cumulants = gridspec.GridSpec(2, 6, width_ratios=[1, 1, 1, 1,1,1], height_ratios=[1, 1])
                              #hspace = 0.3,wspace = 2.3)
 #(1, 2, width_ratios=[6, 4], figure=fig)

# position mean
plt.subplot(gs_cumulants[0, 0:2])
plt.plot(times_t0,  [linear_position_cumulant(t0) for t0 in times_t0],lw=lw)
plt.title('(a)',fontsize = fontsizetitles,x = xtitle,y = ytitle,pad = titlepad,zorder=titlezorder)
ax = format_axes(plt.gca(),fontsize)
ax.set_ylim((-0.05,1.2))
ax.set_ylabel('Position Mean',fontsize = fontsizetitles)

# position variance
plt.subplot(gs_cumulants[0, 2:4])
plt.plot(times_t0,  [position_variance(t0) for t0 in times_t0],lw=lw)
plt.title('(b)',fontsize = fontsizetitles,x = xtitle,pad = titlepad ,y = ytitle,zorder=titlezorder)
ax = format_axes(plt.gca(),fontsize)
ax.set_ylabel('Position Variance',fontsize = fontsizetitles)

# cross corr
plt.subplot(gs_cumulants[0, 4:])
plt.plot(times_t0, [cross_correlation(t0) for t0 in times_t0],lw=lw)
plt.title('(c)',fontsize = fontsizetitles,x = xtitle,pad = titlepad ,y = ytitle,zorder=titlezorder)
ax = format_axes(plt.gca(),fontsize)
ax.set_ylabel('Cross Correlation',fontsize = fontsizetitles)


# momentum mean
plt.subplot(gs_cumulants[1,:3])
plt.plot(times_t0, [mom_mean(t0) for t0 in times_t0],lw=lw)
plt.title('(d)',fontsize = fontsizetitles,x = xtitle*(2/3),pad = titlepad ,y = ytitle,zorder=titlezorder)
ax = format_axes(plt.gca(),fontsize)
ax.set_ylabel('Momentum Mean',fontsize = fontsizetitles)

# momentum variance
mom_var = plt.subplot(gs_cumulants[1, 3:])
plt.plot(times_t0,  [momentum_variance(t0) for t0 in times_t0],lw=lw)
plt.title('(e)',fontsize = fontsizetitles,x = xtitle*(2/3), pad = titlepad ,y = ytitle,zorder=titlezorder)
ax = format_axes(plt.gca(),fontsize)
ax.set_ylabel('Momentum Variance',fontsize = fontsizetitles)

plt.tight_layout()

# Show the plot
plt.savefig(filename, format=format_type)
plt.savefig("kl_land_cumulants.pdf", format="pdf")
plt.savefig("kl_land_cumulants.png", format="png")

plt.close()


###Peak heights

In [ ]:
from scipy.signal import find_peaks

plt.figure()
series = smooth_distribution(5)
peaks,properties = find_peaks(series, prominence=(0.0001, None))


prominences = properties['prominences']

plt.plot(series)
plt.plot(peaks, series[peaks], "x")
plt.fill_between(range(0,len(q_axis)),p_final/pf_norm,alpha= 0.3)

plt.savefig("test_peaks.png")
plt.close()

In [ ]:
c5 = "orange"

height_a = np.zeros(len(times_t0))
height_b = np.zeros(len(times_t0))
ODheight_a = np.zeros(len(times_t0))
ODheight_b = np.zeros(len(times_t0))

for t in enumerate(times_t0):
  series = smooth_distribution(t[1])
  rho_vals = rho(t[1])
  peaks,props  = find_peaks(series,prominence=(0.01, None))
  peaks_rho,props_rho  = find_peaks(rho_vals,prominence=(0.01, None))

  prominences = props['prominences']
  prominences_rho = props_rho['prominences']


  ind1 = peaks[np.argmax(prominences)]
  ind1_rho = peaks_rho[np.argmax(prominences_rho)]
  try:
    ind2 = peaks[np.argsort(prominences)[-2]]
  except:
    ind2 = ind1
  try:
    ind2_rho = peaks_rho[np.argsort(prominences_rho)[-2]]
  except:
    ind2_rho = ind1_rho


  height_a[t[0]] = series[ind1]
  height_b[t[0]] = series[ind2]
  ODheight_a[t[0]] = rho_vals[ind1_rho]
  ODheight_b[t[0]] = rho_vals[ind2_rho]

plt.figure(figsize = (10,6))

plt.plot(times_t0,height_a,lw=lw,color = c1,label = "Underdamped")
plt.plot(times_t0,height_b,lw=lw,color = c1)
plt.plot(times_t0,ODheight_a,lw=lw,color = c5,label = "Underdamped")
plt.plot(times_t0,ODheight_b,lw=lw,color = c5)
#plt.ylim((0.25,0.45))

ax = plt.gca()
format_axes(ax,fontsizeticks)
ax.set_ylabel("Peak Height", fontsize= fontsizetitles)

l0 = mlines.Line2D([], [], color=c1, lw = lw)
l1 = mlines.Line2D([], [], color=c3, lw = lw)
l2 = mlines.Line2D([], [], color="orange", lw = lw)

ax.legend([l0,l2],["Underdamped","Overdamped"],
              prop = {"size": fontsizetitles },
              ncol = 1,
              loc = "upper left",
              #handler_map={tuple: HandlerTuple(ndivide=None)},
              #bbox_to_anchor=(0.1, -0.1),
              frameon = False)


plt.savefig("kl_peakheights.eps")
plt.savefig("kl_peakheights.pdf")
plt.savefig("kl_peakheights.png")
plt.close()

###Expansion and Contraction

Below creates the Figure 5 from the results obtained by calculations in KL Gauss notebooks.

In [ ]:
####expansion/contraction#############

#path = "ep_land20.csv"
path_ex = "kl_exco.txt"
path_con = "kl_contraction.txt"
data_ex = pd.read_csv(path_ex,skiprows = 0,sep =" ")
df_ex = pd.DataFrame(data_ex)
data_con = pd.read_csv(path_con,skiprows = 0,sep =" ")
df_con = pd.DataFrame(data_con)

path_ex0 = "klExpansionUstar 0.txt"
path_con0 = "klContractionUstar0.txt"
data_ex0 = pd.read_csv(path_ex0,skiprows = 1,sep =" ",header =None)
df_ex0 = pd.DataFrame(data_ex0)
data_con0 = pd.read_csv(path_con0,skiprows = 1,sep =" ",header = None)
df_con0 = pd.DataFrame(data_con0)

Taxis = np.linspace(3,10,7)
Taxis_all = np.linspace(1,10,10)

df_ex.columns = ["KL"]
df_con.columns = ["KL"]

df_ex0.columns = ["T","KL"]
df_con0.columns = ["T","KL"]

FileNotFoundError: [Errno 2] No such file or directory: 'kl_exco.txt'

In [ ]:
df_ex_1 = df_ex.drop(index=[1,2,3])
df_con_1 = df_con.drop(index=[1,2,3])

In [ ]:

#set ylim for plot
ymin = -20
ymax = 20


####DISTRIUBUTIONS GO HERE
####KL divergence gauss
fig = plt.figure(figsize = (15,8))
# Create a 1x5 grid with different widths for the bottom row

# t0 distribution
ex1, = plt.plot(Taxis,df_ex_1.KL/4,color=c1,lw=lw,  label =r"Expansion")
con1, = plt.plot(Taxis,df_con_1.KL/4,color=c3,lw=lw,  label =r"Contraction")
#ex0, = plt.plot(Taxis,df_ex0.KL,color=c2,lw=lw,  label =r"Expansion, $U_{\star}=0$")
#con0, = plt.plot(Taxis,df_con0.KL,color=c4,lw=lw,  label =r"Contraction, $U_{\star}=0$")

ax0 = plt.gca()
#format_dist_axes(ax)
ax0.set_ylabel(r'KL Divergence',fontsize = fontsizetitles)
ax0.set_xlabel(r'$T$',fontsize = fontsizetitles)
#ax.set_xticklabels([])
ax0.tick_params(axis='y', labelsize=fontsizeticks)
ax0.tick_params(axis='x', labelsize=fontsizeticks)
ax0.set_xlim((2.8,10))
#plt.yscale('log')#, linthresh=1)
ax0.set_ylim((-0.05,0.4))

ax0.legend( prop = {"size": fontsizetitles })

# Show the plot
plt.savefig("kl_exco.eps", format="eps")
plt.savefig("kl_exco.pdf", format="pdf")
plt.savefig("kl_exco.png", format="png")

#files.download("kl_land_drift.eps")
#files.download("kl_land_drift.pdf")
plt.close()


In [ ]:

#set ylim for plot
ymin = -20
ymax = 20

####DISTRIUBUTIONS GO HERE
####KL divergence gauss
fig = plt.figure(figsize = (15,8))
# Create a 1x5 grid with different widths for the bottom row

# t0 distribution
ex1, = plt.plot(Taxis_all,df_ex.KL/4,color=c1,lw=lw,  label =r"Expansion")
con1, = plt.plot(Taxis_all,df_con.KL/4,color=c3,lw=lw,  label =r"Contraction")
ex0, = plt.plot(Taxis_all,df_ex0.KL/4,color=c2,  label =r"Expansion, $U_{\star}=0$")
con0, = plt.plot(Taxis_all,df_con0.KL/4,color=c4,  label =r"Contraction, $U_{\star}=0$")

ax0 = plt.gca()
#format_dist_axes(ax)
ax0.set_ylabel(r'KL Divergence',fontsize = fontsizetitles)
ax0.set_xlabel(r'$T$',fontsize = fontsizetitles)
#ax.set_xticklabels([])
ax0.tick_params(axis='y', labelsize=fontsizeticks)
ax0.tick_params(axis='x', labelsize=fontsizeticks)
ax0.set_xlim((0.8,10))
plt.yscale('log')#, linthresh=1)
#ax0.set_ylim((-0.05,1.4))

ax0.legend( prop = {"size": fontsizetitles })

# Show the plot
plt.savefig("kl_exco_all.eps", format="eps")
plt.savefig("kl_exco_all.pdf", format="pdf")
plt.savefig("kl_exco_all.png", format="png")

plt.close()


NameError: name 'Taxis_all' is not defined